In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

model = ChatOpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str
    

In [ ]:
def create_outline(state: BlogState) -> BlogState:
    
    # fetch title
    title = state['title']

    prompt = f"generate a detailed outline for a blog on the topic - {title}"

    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [ ]:
def create_blog(state: BlogState) -> BlogState:
      
      title = state['title']
      outline = state['outline']

      prompt = f"Write a detailed blog on the title using the following outline /n {outline}"

      content = model.invoke(prompt).content

      state['content'] = content
      
      return state

In [ ]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, create_outline)
graph.add_edge(create_outline, create_blog)
graph.add_edge(create_blog, END)

In [ ]:
workflow = graph.compile()

initial_state = {'title': 'Rise of AI in india'}

final_state = workflow.invoke(initial_state)

print(final_state)
print(final_state['outline'])
print(final_state['content'])